In [1]:
# This notebook will tack the coron-specific filter boresight offsets from a CSV file
# into the FITS distortion reference file
# 
# From here it can be called by miricoord, pulled out to pipeline ref files, etc.
#
# Offsets will be ZERO for filters in which it hasn't been measured for the coronagraphs,
# including for F770W, which will ensure that we can call miricoord in the usual manner
# and have it work as expected since we're tying F770W to each coron filter

In [2]:
from astropy.io import fits
import numpy as np
from astropy.io import ascii
import miricoord.imager.mirim_tools as mt

In [3]:
distfile=mt.get_fitsreffile()
disthdus=fits.open(distfile)
borehdu=disthdus['Boresight offsets']

In [4]:
fits.info(distfile)

Filename: /Users/dlaw/jwcode/miricoord/data/fits/flt2/MIRI_FM_MIRIMAGE_DISTORTION_SS.05.00.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      41   ()      
  1  B matrix      1 ImageHDU         9   (5, 5)   float64   
  2  A matrix      1 ImageHDU         9   (5, 5)   float64   
  3  T matrix      1 ImageHDU         9   (3, 3)   float64   
  4  M matrix      1 ImageHDU         9   (3, 3)   float64   
  5  BI matrix     1 ImageHDU         9   (5, 5)   float64   
  6  AI matrix     1 ImageHDU         9   (5, 5)   float64   
  7  TI matrix     1 ImageHDU         9   (3, 3)   float64   
  8  MI matrix     1 ImageHDU         9   (3, 3)   float64   
  9  Boresight offsets    1 BinTableHDU     15   15R x 3C   [7A, D, D]   
 10  D matrix      1 ImageHDU         9   (5, 5)   float64   
 11  C matrix      1 ImageHDU         9   (5, 5)   float64   
 12  F matrix      1 ImageHDU         9   (5, 5)   float64   
 13  E matrix      1 ImageHDU      

In [5]:
borehdu.data

FITS_rec([('F560W',  0.08, 0.06), ('F770W',  0.  , 0.  ),
          ('F1000W',  0.14, 0.62), ('F1130W', -0.15, 0.59),
          ('F1280W',  0.08, 0.7 ), ('F1500W',  0.36, 0.74),
          ('F1800W', -0.39, 0.73), ('F2100W',  0.02, 0.27),
          ('F2550WR',  0.27, 1.45), ('F2550W',  0.15, 0.55),
          ('F1065C',  0.6 , 2.15), ('F1140C',  0.42, 1.56),
          ('F1550C',  1.17, 1.47), ('F2300C', -1.35, 2.11),
          ('FND',  0.07, 0.26)],
         dtype=(numpy.record, [('FILTER', 'S7'), ('COL_OFFSET', '>f8'), ('ROW_OFFSET', '>f8')]))

In [6]:
# Make a copy of the boresight HDU for MASK1065, MASK1140, MASK1550, MASKLYOT
borehdu1065=borehdu.copy()
borehdu1065.header['EXTNAME']='BoresightCORON1065'

borehdu1140=borehdu.copy()
borehdu1140.header['EXTNAME']='BoresightCORON1140'

borehdu1550=borehdu.copy()
borehdu1550.header['EXTNAME']='BoresightCORON1550'

borehdulyot=borehdu.copy()
borehdulyot.header['EXTNAME']='BoresightCORONLYOT'

nfilter=len(borehdu.data)
print('nfilter = ',nfilter)

nfilter =  15


In [7]:
# Read the updated coron offsets csv
coron=ascii.read('coronagraph_boresight_offsets-pixels.csv')

In [8]:
coron

subarray,filter,ref_filter,dx,dy
str8,str6,str6,float64,float64
MASK1065,F1065C,F1065C,0.0,0.0
MASK1065,F1500W,F1065C,0.19121024138381648,-0.822911052620391
MASK1065,F560W,F1065C,-0.03309756541382852,-1.4387896999111953
MASK1065,FND,F1065C,0.13582106901312335,-1.2279678857520966
MASK1140,F1140C,F1140C,0.0,0.0
MASK1140,F1500W,F1140C,0.38138752258285713,-0.5872124551206599
MASK1140,F560W,F1140C,0.11230576482675791,-1.2461317298956693
MASK1140,FND,F1140C,0.3271697855783265,-0.9660751864951773
MASK1550,F1500W,F1550C,-0.08808294653695123,-0.7026117757770026


In [9]:
# Populate information in the borehdu1065 HDU
for ii in range(0,nfilter):
    indx=np.where((coron['subarray'] == 'MASK1065')&(coron['filter'] == borehdu1065.data['FILTER'][ii]))[0]
    if (len(indx) == 1):
        borehdu1065.data['COL_OFFSET'][ii]=coron[indx]['dx']
        borehdu1065.data['ROW_OFFSET'][ii]=coron[indx]['dy']
    else:
        borehdu1065.data['COL_OFFSET'][ii]=0
        borehdu1065.data['ROW_OFFSET'][ii]=0

/var/folders/lg/jw_brb093bl9n39fbgn1cw080001fw/T/ipykernel_50789/2624377715.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  borehdu1065.data['COL_OFFSET'][ii]=coron[indx]['dx']
/var/folders/lg/jw_brb093bl9n39fbgn1cw080001fw/T/ipykernel_50789/2624377715.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  borehdu1065.data['ROW_OFFSET'][ii]=coron[indx]['dy']


In [10]:
# Populate information in the borehdu1140 HDU
for ii in range(0,nfilter):
    indx=np.where((coron['subarray'] == 'MASK1140')&(coron['filter'] == borehdu1140.data['FILTER'][ii]))[0]
    if (len(indx) == 1):
        borehdu1140.data['COL_OFFSET'][ii]=coron[indx]['dx']
        borehdu1140.data['ROW_OFFSET'][ii]=coron[indx]['dy']
    else:
        borehdu1140.data['COL_OFFSET'][ii]=0
        borehdu1140.data['ROW_OFFSET'][ii]=0

/var/folders/lg/jw_brb093bl9n39fbgn1cw080001fw/T/ipykernel_50789/3756976466.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  borehdu1140.data['COL_OFFSET'][ii]=coron[indx]['dx']
/var/folders/lg/jw_brb093bl9n39fbgn1cw080001fw/T/ipykernel_50789/3756976466.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  borehdu1140.data['ROW_OFFSET'][ii]=coron[indx]['dy']


In [11]:
# Populate information in the borehdu1550 HDU
for ii in range(0,nfilter):
    indx=np.where((coron['subarray'] == 'MASK1550')&(coron['filter'] == borehdu1550.data['FILTER'][ii]))[0]
    if (len(indx) == 1):
        borehdu1550.data['COL_OFFSET'][ii]=coron[indx]['dx']
        borehdu1550.data['ROW_OFFSET'][ii]=coron[indx]['dy']
    else:
        borehdu1550.data['COL_OFFSET'][ii]=0
        borehdu1550.data['ROW_OFFSET'][ii]=0

/var/folders/lg/jw_brb093bl9n39fbgn1cw080001fw/T/ipykernel_50789/2941484206.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  borehdu1550.data['COL_OFFSET'][ii]=coron[indx]['dx']
/var/folders/lg/jw_brb093bl9n39fbgn1cw080001fw/T/ipykernel_50789/2941484206.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  borehdu1550.data['ROW_OFFSET'][ii]=coron[indx]['dy']


In [12]:
# Populate information in the borehdulyot HDU
for ii in range(0,nfilter):
    indx=np.where((coron['subarray'] == 'MASKLYOT')&(coron['filter'] == borehdulyot.data['FILTER'][ii]))[0]
    if (len(indx) == 1):
        borehdulyot.data['COL_OFFSET'][ii]=coron[indx]['dx']
        borehdulyot.data['ROW_OFFSET'][ii]=coron[indx]['dy']
    else:
        borehdulyot.data['COL_OFFSET'][ii]=0
        borehdulyot.data['ROW_OFFSET'][ii]=0

/var/folders/lg/jw_brb093bl9n39fbgn1cw080001fw/T/ipykernel_50789/2497336618.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  borehdulyot.data['COL_OFFSET'][ii]=coron[indx]['dx']
/var/folders/lg/jw_brb093bl9n39fbgn1cw080001fw/T/ipykernel_50789/2497336618.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  borehdulyot.data['ROW_OFFSET'][ii]=coron[indx]['dy']


In [13]:
# Append new HDUs to the distortion file
newhdus=disthdus.copy()

for key, borehdu in [('BoresightCORON1065', borehdu1065),
                 ('BoresightCORON1140', borehdu1140),
                 ('BoresightCORON1550', borehdu1550),
                 ('BoresightCORONLYOT', borehdulyot)
                 ]:
    try:
        newhdus[key] = borehdu
    except KeyError:
        newhdus.append(borehdu)
        
newhdus.info()

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      41   ()      
  1  B matrix      1 ImageHDU         9   (5, 5)   float64   
  2  A matrix      1 ImageHDU         9   (5, 5)   float64   
  3  T matrix      1 ImageHDU         9   (3, 3)   float64   
  4  M matrix      1 ImageHDU         9   (3, 3)   float64   
  5  BI matrix     1 ImageHDU         9   (5, 5)   float64   
  6  AI matrix     1 ImageHDU         9   (5, 5)   float64   
  7  TI matrix     1 ImageHDU         9   (3, 3)   float64   
  8  MI matrix     1 ImageHDU         9   (3, 3)   float64   
  9  Boresight offsets    1 BinTableHDU     15   15R x 3C   ['7A', 'D', 'D']   
 10  D matrix      1 ImageHDU         9   (5, 5)   float64   
 11  C matrix      1 ImageHDU         9   (5, 5)   float64   
 12  F matrix      1 ImageHDU         9   (5, 5)   float64   
 13  E matrix      1 ImageHDU         9   (5, 5)   float64   
 14  DI matrix

In [14]:
# Update header info
newhdus[0].header['FILENAME']='MIRI_FM_MIRIMAGE_DISTORTION_SS.06.00.fits'
newhdus[0].header['VERSION']='SS.06.00'
newhdus[0].header['HISTORY']='SS.06.00 updates Coron-specific boresight tables'
newhdus[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
TELESCOP= 'JWST    '           / The telescope used to acquire the data         
INSTRUME= 'MIRI    '           / Instrument used to acquire data                
MODELNAM= 'FM      '           / Instrument model name                          
DETECTOR= 'MIRIMAGE'           / String mnemonic for SCA used to acquire data   
DETSETNG= 'N/A     '           / Detector settings parameter                    
EXP_TYPE= 'MIR_IMAGE'          / Exposure type                                  
READPATT= 'N/A     '           / Detector read out pattern parameter set        
SUBARRAY= 'FULL    '           / Det. subarray; istortion applies to full array 
FASTAXIS=                   

In [15]:
new_path = '../../imager/makeboresight/MIRI_FM_MIRIMAGE_DISTORTION_SS.06.00.fits'
newhdus.writeto(new_path, overwrite=True)

In [16]:
borehdu1550.data

FITS_rec([('F560W', -0.55497056, -1.4172281 ),
          ('F770W',  0.        ,  0.        ),
          ('F1000W', -0.329     , -0.855     ),
          ('F1130W',  0.        ,  0.        ),
          ('F1280W',  0.        ,  0.        ),
          ('F1500W', -0.08808295, -0.70261178),
          ('F1800W',  0.        ,  0.        ),
          ('F2100W',  0.        ,  0.        ),
          ('F2550WR',  0.        ,  0.        ),
          ('F2550W',  0.        ,  0.        ),
          ('F1065C',  0.        ,  0.        ),
          ('F1140C',  0.        ,  0.        ),
          ('F1550C',  0.        ,  0.        ),
          ('F2300C',  0.        ,  0.        ),
          ('FND', -0.25904089, -1.08108376)],
         dtype=(numpy.record, [('FILTER', 'S7'), ('COL_OFFSET', '>f8'), ('ROW_OFFSET', '>f8')]))

In [17]:
fits.getdata(new_path, 20)

FITS_rec([('F560W', -0.55497056, -1.4172281 ),
          ('F770W',  0.        ,  0.        ),
          ('F1000W', -0.329     , -0.855     ),
          ('F1130W',  0.        ,  0.        ),
          ('F1280W',  0.        ,  0.        ),
          ('F1500W', -0.08808295, -0.70261178),
          ('F1800W',  0.        ,  0.        ),
          ('F2100W',  0.        ,  0.        ),
          ('F2550WR',  0.        ,  0.        ),
          ('F2550W',  0.        ,  0.        ),
          ('F1065C',  0.        ,  0.        ),
          ('F1140C',  0.        ,  0.        ),
          ('F1550C',  0.        ,  0.        ),
          ('F2300C',  0.        ,  0.        ),
          ('FND', -0.25904089, -1.08108376)],
         dtype=(numpy.record, [('FILTER', 'S7'), ('COL_OFFSET', '>f8'), ('ROW_OFFSET', '>f8')]))